In [1]:
import pandas as pd
import tarfile
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse, parseString
from collections import Counter
import numpy as np
import re
import csv
from tqdm import tqdm
import csv
import pickle
import matplotlib.pyplot as plt
import time
import os
import xmltodict
from collections.abc import MutableMapping
import copy

In [2]:
#pandas settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
def tar_extract_and_populate(location_input: str):
    """This function takes in a set of tar files, extracts them in a directory of choice and populates a list of files in the directory"""
    
    tar_data_files = [name for name in os.listdir(location_input)]
    xml_files_list=[]
    year = location_input[-4:]
    
    for tar_data_file in tar_data_files:
        #open the tar.gz file as a tar archive
        print("{}/{}".format(location_input, tar_data_file))
        tar_archive = tarfile.open(name="{}/{}".format(location_input, tar_data_file), mode="r:gz")

        #extract the tar archive in a directory of choice
        tar_archive.extractall("../new_data/extracted_data/")

        for file in tar_archive.getnames():
            if file[-4:] == ".xml":
                xml_files_list.append("{}/{}".format(location_input, file))

        tar_archive.close()

    return xml_files_list

In [4]:
#directory = "new_data/raw_data"
#xml_files_list = tar_extract_and_populate(directory)
#with open("../3.new_data/files_list.pickle", "wb") as f:
#    pickle.dump(new_file_names, f)

In [5]:
with open("../3.new_data/files_list.pickle", "rb") as f:
    files_list = pickle.load(f)

In [6]:
def flatten_dict(d, parent_key ='', sep ='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
 
        if isinstance(v, MutableMapping):
            items.extend(flatten_dict(v, new_key, sep = sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
data_dict = {}
for i, file_path in tqdm(enumerate(files_list)):

    with open(file_path, "r", encoding="utf-8", errors="ignore") as fileptr:
        xml_content = fileptr.read()

    tender_dict = xmltodict.parse(xml_content, process_namespaces=False)
    tender_flat_dict = flatten_dict(tender_dict)
    data_dict[i] = tender_flat_dict

In [ ]:
#with open("new_data/data_dict_440K_460K", "wb") as f:
#    pickle.dump(data_dict, f)

In [7]:
#LOAD AND COMBINE ALL DATA
directory = "../3.new_data/"
data_dict_files = [name for name in os.listdir(directory) if "data_dict_" in name]
data_dict_files

['data_dict_0_20k',
 'data_dict_120K_160K',
 'data_dict_160K_200K',
 'data_dict_200K_240K',
 'data_dict_20k_40k',
 'data_dict_240K_280K',
 'data_dict_280K_320K',
 'data_dict_320K_360K',
 'data_dict_360K_400K',
 'data_dict_400K_440K',
 'data_dict_40k_80K',
 'data_dict_440K_480K',
 'data_dict_460K_480K',
 'data_dict_480K_520K',
 'data_dict_520K_560K',
 'data_dict_560K_600k',
 'data_dict_600k_',
 'data_dict_80K_120K']

In [8]:
file = "data_dict_0_20k"
with open("../3.new_data/{}".format(file), "rb") as f:
    data_dict = pickle.load(f)

In [9]:
base = r'TED_EXPORT_FORM_SECTION_F\d{2}_2014_OBJECT_CONTRACT_OBJECT_DESCR'

short_descr_pattern = r'TED_EXPORT_FORM_SECTION_F\d{2}_2014_OBJECT_CONTRACT_SHORT_DESCR_P(?!_.*)'
object_descr_pattern_lots = base + r'(?!_.*)'

duration_type_pattern = base + r'_DURATION_@TYPE'
duration_text_pattern = base + r'_DURATION_#text'
title_text_pattern = r'TED_EXPORT_FORM_SECTION_F03_2014_OBJECT_CONTRACT_TITLE_P'
doc_id_pattern = r'0*([1-9][0-9]*|0[1-9][0-9]*)'
date_start_pattern = base + r'_DATE_START'
date_end_pattern = base + r'_DATE_END'

ac_criterion_pattern = base + r'(_AC_AC_QUALITY|_AC_AC_COST)(_AC_CRITERION)?'
ac_criterion_pattern_lots = r'(AC_AC_QUALITY|AC_AC_COST)'

#OBJECT_CONTRACT_OBJECT_DESCR
val_object_pattern = base + r'_VAL_OBJECT_#text'
val_object_currency_pattern = base + r'_VAL_OBJECT_@CURRENCY'

#OBJECT_CONTRACT
base_val = r'TED_EXPORT_FORM_SECTION_F\d{2}_2014_OBJECT_CONTRACT_VAL_'
val_estimated_pattern = base_val + r'ESTIMATED_TOTAL_#text'
val_estimated_currency_pattern = base_val + r'ESTIMATED_TOTAL_@CURRENCY'
val_estimated_high_pattern = base_val + r'RANGE_TOTAL_HIGH'
val_estimated_low_pattern = base_val + r'RANGE_TOTAL_LOW'
val_total_pattern = base_val + r'TOTAL_#text'
val_total_currency_pattern = base_val + r'TOTAL_@CURRENCY'


#
base_award = r'TED_EXPORT_FORM_SECTION_F\d{2}_2014_AWARD_CONTRACT_AWARDED_CONTRACT_VALUES_VAL_'
award_val_estimated_pattern = base_award + r'ESTIMATED_TOTAL_#text'
award_val_estimated_currency_pattern = base_award + r'ESTIMATED_TOTAL_@CURRENCY'

award_val_total_pattern = base_award + r'TOTAL_#text'
award_val_total_currency_pattern = base_award + r'TOTAL_@CURRENCY'

award_val_high_pattern = base_award + r'RANGE_TOTAL_HIGH'
award_val_low_pattern = base_award + r'RANGE_TOTAL_LOW'

In [ ]:
#get the row and short description out of the dict
extracted_data = []

#TED_EXPORT_FORM_SECTION_F03_2014_OBJECT_CONTRACT_SHORT_DESCR_P
count = 0
count_nested_dicts_in_list = 0

#GO THROUGH ALL THE FILES
for file in tqdm(list(data_dict_files)):
    with open("../3.new_data/{}".format(file), "rb") as f:
        data_dict = pickle.load(f)
        
        #WITHIN EACH FILE, LOOP THROUGH EACH TENDER DICT
        for i, file_number in enumerate(list(data_dict.keys())):
            
            #INITILIAZE THE DICT THAT IS TO BE FILLED WITH INFORMATION
            tender = {"ID_NOTICE_CAN": "",
                    "LG_ORIG": "",
                    "short description": "",
                    "duration type": "",
                    "duration": "",
                    "title": "",
                    "date start": "",
                    "date end": "",
                    "ac criterion": "",

                    "value object": "",
                    "value object currency": "",
                    
                    "obj_contract value estimated": "",
                    "obj_contract value estimated currency": "",
                    "obj_contract value estimated high": "",
                    "obj_contract value estimated low": "",
                    "obj_contract value total": "",
                    "obj_contract value total currency": "",
                    
                    "award value estimated": "",
                    "award value estimated currency": "",
                    "award value total": "",
                    "award value total currency": "",
                    "award value high": "",
                    "award value low": ""}
            
            criteria_text = ""

            #IF A TAG MATCHES SOME DEFINITION, STORE THE INFORMATION
            for tag in data_dict[file_number].keys():
                if "@DOC_ID" in tag:
                    doc_id_list = str(data_dict[file_number][tag]).split("-")
                    doc_id_number = re.findall(doc_id_pattern, doc_id_list[0])
                    doc_id = doc_id_list[1] + doc_id_number[0]
                    tender["ID_NOTICE_CAN"] = doc_id
                
                elif "LG_ORIG" in tag:
                    if type(data_dict[file_number][tag]) == list:
                        tender["LG_ORIG"] = data_dict[file_number][tag][0]
                    else:
                        tender["LG_ORIG"] = data_dict[file_number][tag]
                
                elif re.match(short_descr_pattern, tag):
                    if type(data_dict[file_number][tag]) == list:
                        if type(data_dict[file_number][tag][0]) == str:
                            tender["short description"] = data_dict[file_number][tag][0]
                        else:
                            count_nested_dicts_in_list += 1
                    else:
                        tender["short description"] = data_dict[file_number][tag]  
                elif re.match(title_text_pattern, tag):
                    tender["title"] = data_dict[file_number][tag]
                elif re.match(ac_criterion_pattern, tag) and "WEIGHTING" not in tag:
                    if type(data_dict[file_number][tag]) == list:
                        for criteria_dict in data_dict[file_number][tag]:
                            if criteria_dict["AC_CRITERION"] not in criteria_text:
                                criteria_text = criteria_text + criteria_dict["AC_CRITERION"] + " "
                            else:
                                continue
                    else:
                        criteria_text = criteria_text + data_dict[file_number][tag] + " "
                elif re.match(duration_type_pattern, tag):
                    tender["duration type"] = data_dict[file_number][tag]
                elif re.match(duration_text_pattern, tag):
                    tender["duration"] = data_dict[file_number][tag]
                elif re.match(date_start_pattern, tag):
                    tender["date start"] = data_dict[file_number][tag]
                elif re.match(date_end_pattern, tag):
                    tender["date end"] = data_dict[file_number][tag]
                
                #object values
                elif re.match(val_object_pattern, tag):
                    tender["value object"] = data_dict[file_number][tag]
                elif re.match(val_object_currency_pattern, tag):
                    tender["value object currency"] = data_dict[file_number][tag]

                #object_contract_val
                elif re.match(val_estimated_pattern, tag):
                    tender["obj_contract value estimated"] = data_dict[file_number][tag]
                elif re.match(val_estimated_currency_pattern, tag):
                    tender["obj_contract value estimated currency"] = data_dict[file_number][tag]
                elif re.match(val_estimated_high_pattern, tag):
                    tender["obj_contract value estimated high"] = data_dict[file_number][tag]
                elif re.match(val_estimated_low_pattern, tag):
                    tender["obj_contract value estimated low"] = data_dict[file_number][tag]
                elif re.match(val_total_pattern, tag):
                    tender["obj_contract value total"] = data_dict[file_number][tag]
                elif re.match(val_total_currency_pattern, tag):
                    tender["obj_contract value total currency"] = data_dict[file_number][tag]

                #award values 
                elif re.match(award_val_estimated_pattern, tag):
                    tender["award value estimated"] = data_dict[file_number][tag]
                elif re.match(award_val_estimated_currency_pattern, tag):
                    tender["award value estimated currency"] = data_dict[file_number][tag]
                elif re.match(award_val_total_pattern, tag):
                    tender["award value total"] = data_dict[file_number][tag]
                elif re.match(award_val_total_currency_pattern, tag):
                    tender["award value total currency"] = data_dict[file_number][tag]
                elif re.match(award_val_high_pattern, tag):
                    tender["award value high"] = data_dict[file_number][tag]
                elif re.match(award_val_low_pattern, tag):
                    tender["award value low"] = data_dict[file_number][tag]

                #FOR EACH LOT IN THE TENDER, COPY MAIN DICT, AND FILL WITH INFORMATION PER LOT 
                elif re.match(object_descr_pattern_lots, tag) and type(data_dict[file_number][tag]) == list:
                    count += 1
                    for lot_number in range(0, len(data_dict[file_number][tag])):
                        base_dict_checkpoint = copy.deepcopy(tender)
                        flat_lot_notice = flatten_dict(data_dict[file_number][tag][lot_number])
                        criteria_text_lot = "" #define the criteria at the lot level allow concatenation

                        for variable in flat_lot_notice.keys():
                            if variable == "SHORT_DESCR_P":
                                if type(flat_lot_notice["SHORT_DESCR_P"]) == list:
                                    if type(flat_lot_notice["SHORT_DESCR_P"][0]) == str:
                                        base_dict_checkpoint["short description"] = flat_lot_notice["SHORT_DESCR_P"][0]
                                    else:
                                        count_nested_dicts_in_list +=1
                                else:
                                    base_dict_checkpoint["short description"] = flat_lot_notice["SHORT_DESCR_P"]
                            elif variable == "LOT_NO":
                                base_dict_checkpoint["ID_LOT"] = flat_lot_notice[variable] #str(lot_number)
                            elif "DURATION_@TYPE" in variable:
                                base_dict_checkpoint["duration type"] = flat_lot_notice[variable]
                            elif "DURATION_#text" in variable:
                                base_dict_checkpoint["duration"] = flat_lot_notice[variable]
                            elif "DATE_START" in variable:
                                base_dict_checkpoint["date start"]
                            elif "DATE_END" in variable:
                                base_dict_checkpoint["date end"]
                            elif "TITLE_P" in variable:
                                base_dict_checkpoint["title"] = flat_lot_notice[variable]
                            elif re.match(ac_criterion_pattern_lots, variable)  and "WEIGHTING" not in variable:
                                if type(flat_lot_notice[variable]) == list:
                                    for criteria_dict in flat_lot_notice[variable]:
                                        if criteria_dict["AC_CRITERION"] not in criteria_text_lot:
                                            criteria_text_lot = criteria_text_lot + criteria_dict["AC_CRITERION"] + " "
                                        else:
                                            continue
                                    base_dict_checkpoint["ac criterion"] = criteria_text_lot

                                elif flat_lot_notice[variable] != None:
                                    criteria_text_lot = criteria_text_lot + flat_lot_notice[variable] + " "
                                    base_dict_checkpoint["ac criterion"] = criteria_text_lot
                            else:
                                continue

                        #ADD NaN FOR MISSING VALUES
                        for variable in base_dict_checkpoint.keys():
                            if base_dict_checkpoint[variable] == None:
                                base_dict_checkpoint[variable] = np.nan
                            elif len(base_dict_checkpoint[variable]) < 1:
                                base_dict_checkpoint[variable] = np.nan
                            else:
                                continue

                        extracted_data.append(base_dict_checkpoint)

                    tender = {}
                else:
                    continue

            if len(tender) > 0:
                for variable in tender.keys():
                    if len(tender[variable]) < 1:
                        tender[variable] = np.nan
                    else:
                        continue
                
                tender["ID_LOT"] = -1
                tender["ac criterion"] = criteria_text
                extracted_data.append(tender)

In [ ]:
#SAVE DICT
#with open("../3.new_data/extracted_data_dict_id.pickle", "wb") as f:
#    pickle.dump(extracted_data, f)

#MAKE DF FROM DICT
df = pd.DataFrame.from_records(extracted_data)

#SAVE DF
#with open("../3.new_data/df_extracted_id_xml.pickle", "wb") as f:
#    pickle.dump(df, f)